In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
app = Flask(__name__)

# Path to the CSV file storing the local news
CSV_FILE_PATH = '/content/drive/MyDrive/W6998-DL/Project/local_news_api/local_news.csv'


In [4]:
@app.route('/news', methods=['GET'])
def get_news():
    # Extract city name and date from query parameters
    cityname = request.args.get('cityname')
    date = request.args.get('date')

    # Validate inputs
    if not cityname or not date:
        return jsonify({"error": "Both 'cityname' and 'date' are required parameters."}), 400

    # Check if the file exists
    if not os.path.exists(CSV_FILE_PATH):
        return jsonify({"error": "News data file not found on the server."}), 500

    # Load the CSV file into a DataFrame
    try:
        news_df = pd.read_csv(CSV_FILE_PATH)
        print (news_df)
    except Exception as e:
        return jsonify({"error": f"Failed to read the news data file: {str(e)}"}), 500

    # Filter the DataFrame by city name and date
    #filtered_news = news_df[(news_df['cityname'] == cityname) & (news_df['date'] == date)]
    filtered_news = news_df[(news_df['new york'] == cityname) & (news_df['date'] == date)]

    if filtered_news.empty:
        return jsonify({"message": "No news found for the given city name and date."}), 404

    # Convert the filtered rows to a list of dictionaries
    news_list = filtered_news.to_dict(orient='records')

    return jsonify({"news": news_list}), 200


In [5]:
@app.route('/news', methods=['POST'])
def add_news():
    # Get the data from the request body
    try:
        news_data = request.get_json()
        cityname = news_data['cityname']
        date = news_data['date']
        headline = news_data['headline']
        content = news_data['content']
    except (KeyError, TypeError):
        return jsonify({"error": "Invalid request body. Required fields: 'cityname', 'date', 'headline', 'content'."}), 400

    # Create a new record
    new_record = {
        'cityname': cityname,
        'date': date,
        'headline': headline,
        'content': content
    }

    # Check if the file exists
    file_exists = os.path.exists(CSV_FILE_PATH)

    # Append the new record to the CSV file
    try:
        if file_exists:
            news_df = pd.read_csv(CSV_FILE_PATH)
            news_df = news_df.append(new_record, ignore_index=True)
        else:
            news_df = pd.DataFrame([new_record])

        news_df.to_csv(CSV_FILE_PATH, index=False)
    except Exception as e:
        return jsonify({"error": f"Failed to save the news data: {str(e)}"}), 500

    return jsonify({"message": "News added successfully."}), 201


In [6]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [5]:
    # Extract city name and date from query parameters
    cityname = 'new york'
    date = '2024-11-29'

    news_df = pd.read_csv(CSV_FILE_PATH)
    #print(news_df)

    # Filter the DataFrame by city name and date
    #filtered_news = news_df[(news_df['cityname'] == cityname) & (news_df['date'] == date)]
    filtered_news = news_df[(news_df['cityname'] == cityname) & (news_df['date'] == date)]
    print(filtered_news)



   cityname        date                                               news
1  new york  2024-11-29  Ford’s new electric Puma Gen-E looks like a mi...
2  new york  2024-11-29      OpenAI sued by Canada’s biggest media outlets
